<!--
03/11
Archivos binarios. Pickles.
Apareo de archivos secuenciales.
CLASE DE LABORATORIO 
(Gonzalo)
-->

# Archivos

Por lo general, cuando se trabaja con un archivo se hacen tres operaciones seguidas:

1. Abrir el archivo
2. Procesar el archivo
3. Cerrar el archivo

Y hay que tener cuidado, porque si ocurre algún error con el archivo en algún punto de su procesamiento es necesario encargarse de cerrarlo, antes de que la excepción siga subiendo niveles.

## Trabajando con archivo de una forma segura

Para trabajar con los archivos de una forma más simple es que se agregó la sentencia **with** que define un contexto dentro del cual nos asegura que, ocurra una excepción o no, el archivo se cerrará al momento de salir de ese contexto:

In [3]:
with open('ejemplo.txt') as archivo:
    print '¿El archivo se encuentra abierto?: {}'.format(archivo.closed)
    print
    for linea in archivo:
        longitud = len(linea[:-1])
        print '%2d: %s' % (longitud, linea[:-1])

print
print '¿El archivo se encuentra abierto?: {}'.format(archivo.closed)

¿El archivo se encuentra abierto?: False

70: Python was created in the early 1990s by Guido van Rossum at Stichting
68: Mathematisch Centrum (CWI, see http://www.cwi.nl) in the Netherlands
64: as a successor of a language called ABC.  Guido remains Python's
70: principal author, although it includes many contributions from others.
 0: 
66: In 1995, Guido continued his work on Python at the Corporation for
70: National Research Initiatives (CNRI, see http://www.cnri.reston.va.us)
61: in Reston, Virginia where he released several versions of the
 9: software.
 0: 
64: In May 2000, Guido and the Python core development team moved to
70: BeOpen.com to form the BeOpen PythonLabs team.  In October of the same
62: year, the PythonLabs team moved to Digital Creations (now Zope
68: Corporation, see http://www.zope.com).  In 2001, the Python Software
62: Foundation (PSF, see http://www.python.org/psf/) was formed, a
66: non-profit organization created specifically to own Python-related
66: Inte

Si bien no cerramos explícitamente el archivo usando la función close, al salir del bloque de código que encierra el with el archivo se encontrará cerrado.

## Pickles

Los [*pickles*](https://docs.python.org/2/library/pickle.html#module-pickle) son una forma de guardar estructuras de datos complejas y recuperarlas fácilmente, sin necesidad de convertirlas a texto y luego parsearlas:

### Ejemplo 1: Guardar de a un elemento

Se puede usar los pickles como se hacía con los viejos archivos de Pascal, donde se guardaba un registro detrás del otro; pero con la diferencia de que en este caso no es necesario que todos los registros sean del mismo tipo:

In [4]:
import pickle  # Importo la biblioteca necesaria

# Creo la variable archivo
with open('ejemplo.pkl', 'wb') as archivo:
    pkl = pickle.Pickler(archivo)  # Creo mi punto de acceso a los datos a partir del archivo

    lista1 = [1, 2, 3]
    lista2 = [4, 5]
    diccionario = {'campo1': 1, 'campo2': 'dos'}

    pkl.dump(lista1)         # Guardo la lista1 de [1, 2, 3]
    pkl.dump(None)           # Guardo el valor None
    pkl.dump(lista2)
    pkl.dump('Hola mundo')
    pkl.dump(diccionario)
    pkl.dump(1)

Para leer de un archivo pickle no puedo usar el método readline que usa la estructura for, por lo que no me queda otra que siempre intentar leer hasta que lance una excepción del tipo *EOFError*:

In [7]:
with open('ejemplo.pkl', 'rb') as archivo:
    seguir_leyendo = True
    while seguir_leyendo:
        try:
            data = pickle.load(archivo)  # Leo del archivo un elemento
            print data
        except EOFError:
            seguir_leyendo = False


[1, 2, 3]
None
[4, 5]
Hola mundo
{'campo1': 1, 'campo2': 'dos'}
1


### Ejemplo 2: Guardo una lista de elementos

Así como en el ejemplo anterior guardamos de a un elemento por vez, también podríamos guardar una lista completa que tenga todos los elementos en memoria. De ésta forma, los archivos podrían usarse para cargar los datos al comenzar el programa y guardarlos todos juntos antes de terminar. <br>
Suponiendo que estoy desarrollando un juego en que no van a haber muchos jugadores compitiendo entre si, podría tener una lista con los puntajes y hacer:

In [13]:
lista = [  # Creo la lista que quiero guardar
    {'usuario': 'csanchez', 'puntaje': 5}, 
    {'usuario': 'pperez', 'puntaje': 3}, 
    {'usuario': 'jromero', 'puntaje': 1}, 
]

# Guardo la lista en el archiv
with open('ejemplo_2.pkl', 'wb') as archivo:
    pkl = pickle.Pickler(archivo)
    pkl.dump(lista)

Y si ahora quiero sumarle 3 puntos a un usuario en particular tendría que:

1. Leer todo el archivo en una lista
2. Buscar el usuario y actualizarle los puntos
3. Guardar toda la lista en el archivo


In [14]:
from pprint import pprint

# Leo del archivo
with open('ejemplo_2.pkl', 'rb') as archivo:
    lista_puntajes = pickle.load(archivo)


# Actualizo el puntaje
print 'La lista antes de hacer el cambio es:'
pprint(lista_puntajes)

pos =  next(
    index 
    for (index, d) in enumerate(lista_puntajes) 
    if d["usuario"] == "csanchez")

lista_puntajes[pos]['puntaje'] += 3

print 'La lista una vez hecho el cambio es:'
pprint(lista_puntajes)

# Guardo la lista en el archiv
with open('ejemplo_2.pkl', 'wb') as archivo:
    pkl = pickle.Pickler(archivo)
    pkl.dump(lista_puntajes)

La lista antes de hacer el cambio es:
[{'puntaje': 5, 'usuario': 'csanchez'},
 {'puntaje': 3, 'usuario': 'pperez'},
 {'puntaje': 1, 'usuario': 'jromero'}]
La lista una vez hecho el cambio es:
[{'puntaje': 8, 'usuario': 'csanchez'},
 {'puntaje': 3, 'usuario': 'pperez'},
 {'puntaje': 1, 'usuario': 'jromero'}]


Si bien es muy práctica esta alternativa, tiene el gran inconveniente de no hacer un uso eficiente de la memoria. <br>
Si el archivo contiene millones de usuarios, los estaríamos levantando todos a memoria, con el gran costo que tiene eso (no sólo en espacio, sino también en tiempo) con el único objetivo de sumarle 3 puntos a un único usuario. Y una vez que actualizamos el puntaje de ese usuario, tendríamos que volver a guardar todo el archivo en el disco.

## Abstrayendonos del uso de los pickles

Si bien el uso de los pickles puede resultar muy útil, la forma de leer la información guardada en ellos no suele ser muy cómoda. Por lo que podríamos implementar en un archivo `utils.py` las siguientes dos funciones para abstraernos un poco de cómo se accede a los datos:

```Python
def guardar_en_archivo(archivo, contenido):
    """Guarda lo que le pasen como segundo parámetro en el archivo 
    que recibe como primer parámetro.
    El parámetro llamado archivo tiene que estar abieto en modo 
    binario y para escritura (wb)
    """
    pickler = pickle.Pickler(archivo)
    pickler.dump(contenido)


def leer_desde_archivo(archivo):
    """Lee del archivo archivo un registro y lo retorna junto con una
    variable booleana que indica si llegó al fin de archivo o no.
    El parámetro llamado archivo tiene que estar abieto en modo 
    binario y para lectura (rb).
    Si se intenta leer más allá del fin de archivo, data valdrá None
    y fin_de_archivo será True. En cualquier otro caso fin_de_archivo
    será False.
    """
    try:
        data = pickle.load(archivo)
        fin_de_archivo = False
    except EOFError:
        data = None
        fin_de_archivo = True
    return data, fin_de_archivo

```

En este caso, se podría considerar que la función `leer_desde_archivo` funciona similar a cómo lo hacen los archivos con un registro centinella. <br>
Por lo que podríamos usar:

In [28]:
import utils

curso = [
    {'nombre': 'Sanchez, Lucas', 'nota': 8, 'padron': 90431, 'grupo': 1},
    {'nombre': 'Gigliotti, Emanuel', 'nota': 2, 'padron': 92953, 'grupo': 1},
    {'nombre': 'Aimar, Pablo', 'nota': 10, 'padron': 92407, 'grupo': 1},
    {'nombre': 'Alario, Lucas', 'nota': 8, 'padron': 96556, 'grupo': 2},
    {'nombre': 'Funes Mori, Rodrigo', 'nota': 7, 'padron': 92143, 'grupo': 2},
    {'nombre': 'Funes Mori, Javier', 'nota': 9, 'padron': 92431, 'grupo': 2},
    {'nombre': 'Aimar, Lucas', 'nota': 4, 'padron': 98306, 'grupo': 3},
    {'nombre': 'Sanchez, Carlos', 'nota': 8, 'padron': 97972, 'grupo': 3},
    {'nombre': 'Gago, Fernando', 'nota': 3, 'padron': 93108, 'grupo': 4},
    {'nombre': 'Kranneviter, Matias', 'nota': 5, 'padron': 96739, 'grupo': 5},
]

print 'Creo el archivo vacío usando el modo "wb"'
print 'Si tenía algo, ya lo borre...'
with open('curso.pkl', 'wb') as archivo:
    for alumno in curso:
        print 'Guardando el alumno {} en el archivo'.format(alumno)
        utils.guardar_en_archivo(archivo, alumno)

print
print 'Abro el archivo en modo lectura...'
with open('curso.pkl', 'rb') as archivo:
    alumno, seguir_leyendo = utils.leer_desde_archivo(archivo)
    while seguir_leyendo:
        print 'Leyendo el alumno {} en el archivo'.format(alumno)
        alumno, seguir_leyendo = utils.leer_desde_archivo(archivo)


Creo el archivo vacío usando el modo "wb"
Si tenía algo, ya lo borre...
Guardando el alumno {'nombre': 'Sanchez, Lucas', 'grupo': 1, 'nota': 8, 'padron': 90431} en el archivo


NameError: global name 'pickle' is not defined

## JSON

Otra forma de guardar datos estructurados es usar un módulo llamado [json](https://docs.python.org/2/tutorial/inputoutput.html#saving-structured-data-with-json) y para esto se usan las funciones [dump](https://docs.python.org/2/library/json.html#json.dump) y [load](https://docs.python.org/2/library/json.html#json.load).

# Ejercicios

1. Suponiendo que existe un archivo llamado utils.py donde se encuentran las funciones:

```Python
def guardar_en_archivo(archivo, contenido):
    """Guarda lo que le pasen como segundo parámetro en el archivo que
    recibe como primer parámetro.
    archivo tiene que estar abieto en modo binario y para escritura (wb)
    """
    ...


def leer_desde_archivo(archivo):
    """Lee del archivo archivo un registro y lo retorna junto con una
    variable booleana que indica si llegó al fin de archivo o no.
    archivo tiene que estar abieto en modo binario y para lectura (rb)
    """
    ...
    return data, fin_de_archivo
```
Leer dos archivos (61_matematica.dat y 75_computacion.dat) que tendrán registros con los campos:
    * padron
    * nombre
    * apellido
    * nota
    * codigo_departamento
    * codigo_materia
y armar uno nuevo donde sólo figuren las notas de los alumnos aprobados ordenados por padrón.<br>
Ambos archivos están ordenados por padrón y se deben leer una única vez. Como los archivos pueden ser muy grandes, no se pueden guardar en memoria.<br>
Una vez procesados los dos archivos se tienen que informar, para cada materia, cuántos alumnos aprobaron y cuántos desaprobaron.